# Step 4: Preprocessing & Feature Engineering

Load the final data from steps 2 & 3, create our target variable (and any other relevant features), then scale and prepare data for modelling.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
# Load final data
data = pd.read_csv('data/processed/data.csv')
data.head()

In [ ]:
data.columns

In [ ]:
data.drop(['Unnamed: 0','customer.1'], inplace=True, axis=1)
data.info()

## 4.1 Set Index

In [ ]:
# Index will be multi-index: customer and invoice date
data['invoice_date'] = pd.to_datetime(data['invoice_date'])
data.set_index(['customer','invoice_date'], inplace=True)
data.head()

## 4.2 Create Target Feature

The target feature will be a y/n - was this customer's purchase of a higher priced item than any previous purchases. There will be several steps towards creating this:

* Creating a ranking of items by average price - different customers have different purchase prices, so using the average price paid should help even that out.

* Using that information to add a column with "highest lifetime item" - this will list the highest priced item that customer has purchased in their history to that point.

### 4.2.1 Create ranking of items by average price

In [ ]:
# What's the spread of number of items sold?

# Create a series with the number of items sold by item
item_count = data.groupby('description')['item_price'].count().to_frame()
item_count.sort_values('item_price', ascending=False, inplace = True)
item_count.columns = ['num_times_sold']
item_count.head(10)

In [ ]:
# Find a threshhold below-which it makes sense to drop
sns.countplot(item_count['num_times_sold'])
plt.title('Spread of number of items sold by item')
plt.xlabel('Total # of item sold')
plt.ylabel('Count of the number of items sold per quantity')

plt.tight_layout()
plt.savefig('figures/item_countplot.png')

From the above plot, there are lots of items that only sold once. Those we don't necessarily want, but this doesn't give us any clear "cut off" point. Let's cut off items only sold once at this point.

In [ ]:
# Drop items only sold once
item_count = item_count[item_count['num_times_sold']>1]

In [ ]:
# Drop 'Rental' and 'Misc Hardware'
item_count.drop(['RENTAL','MISCELLANEOUS HARDWARE'], inplace=True)

### Use that list of items to drop from original data list

In [ ]:
# Make a list of the items to keep (the index from item_count)
keep_list = list(item_count.index)

# Use that to filter the data df and keep only those items (description isin list)
data = data[data['description'].isin(keep_list)]
data.info()

### Use this updated list to calculate and rank items by average price

In [ ]:
items = data.groupby('description')['item_price'].mean().to_frame()
items.columns = ['avg_sale_price']
items.sort_values('avg_sale_price', ascending=False, inplace=True)

# Create top lists for examination
top_25 = items.head(25)
top_50 = items.head(50)
top_100 = items.head(100)

In [ ]:
# Plot the top 25 list
sns.barplot(x=top_25.index, y=top_25.avg_sale_price)
plt.title('Avg Sales Price for Top 25 Priced Items')
plt.ylabel('Avg Sale Price')
plt.xticks(rotation=85)
plt.xlabel('Item Name')

plt.savefig('figures/top25_items.png')

#### Drop "Payment" items
These payment items are almost all the highest priced items and throw things off significantly. Drop anything with "payment" in the name"

In [ ]:
### Drop any item with "payment" in the name
items.reset_index(inplace=True)

items = items[~items['description'].str.contains('PAYMENT')]
items.set_index('description', inplace=True)
items.head()

# Create top lists for examination
top_25 = items.head(25)
top_50 = items.head(50)

In [ ]:
# Plot the top 25 list
sns.barplot(x=top_25.index, y=top_25.avg_sale_price)
plt.title('UPDATED - Avg Sales Price for Top 25 Priced Items')
plt.ylabel('Avg Sale Price')
plt.xticks(rotation=85)
plt.xlabel('Item Name')

plt.savefig('figures/top25_items_no_payment_plans.png')

In [ ]:
# Create a new column that is the item rank
items.reset_index(inplace=True)
items['item_price_rank'] = [i+1 for i in items.index]
items.head()

#### Drop really low priced items too

How to define "really low?"

In [ ]:
items.tail(200)

The last 200 items are still largely accessories - all priced under $40. That seems like a good enough threshold for now. We can always adjust this later.

In [ ]:
# Drop all items priced under $40
items = items[items['avg_sale_price']>40]
items.tail()

### 4.2.2 Keep only the items left in original dataframe

In [ ]:
keep_list = list(items.description.unique())
data = data[data['description'].isin(keep_list)]
data.info()

### 4.2.3 Join the item_price_rank with data df


In [ ]:
# Join item sales rank with data
data = pd.merge(data.reset_index(), items[['description', 'item_price_rank']], 
                on='description', how='inner')
data.info()

In [ ]:
data.head()

### 4.2.4 Drop all but highest priced item for each sale date

Each row of my final data frame will represent the highest ranked item purchased on that sale date by that customer

In [ ]:
# Drop all but the highest priced item for each sale date/customer combo

# Make sure data are sorted by customer, then invoice_date, then item_price descending
data.sort_values(by=['customer','invoice_date','item_price'], ascending=False, inplace=True)

# Drop duplicate (customer/invoice_date) combos, keeping the first (i.e. highest priced)
data.drop_duplicates(subset=['customer','invoice_date'], keep='first', inplace=True)

# Reset index
data.set_index(['customer','invoice_date'], inplace=True)

# Confirm only one item per date per customer
data.sort_index(level='customer')
data.head()

### 4.2.5 Drop unnessecary columns

Since we only needed the item_price to create this ranking, we can drop that column now. (It would be highly correlated with item_price_rank anyway.

In [ ]:
data.drop('item_price',axis=1, inplace=True)

### 4.2.6 Create the Target Feature Column

Now we're ready to create our target feature, which addressing the question: is this the highest priced item purchased by this customer to date?

(Another way of saying, is this the highest ranked item by sales price)

In [ ]:
df = data[['item_price_rank']]
df.reset_index(inplace=True)
df.sort_values(['customer','invoice_date'], ascending=[False, True], inplace=True)
df.head()

In [ ]:
# Loop through the invoice dates for each customer & assign 1 to any purchase ranked higher than
# prior purchases

# this will be the list that becomes target feature
target_list = []

# Loop through each customer
for customer in df.customer.unique():
    cust_df = df[df['customer']==customer]
    
    # counter
    i = 0
    
    for date in cust_df.invoice_date.values:
        # If first item purchased, assign 0
        if i == 0:
            target_list.append(0)
            i += 1
            continue
        
        # For susequent purchases, was this item higher ranked than any previous
        item_max = cust_df.item_price_rank.iloc[:i-1].max()
        
        if cust_df.item_price_rank.iloc[i] > item_max:
            target_list.append(1)
            i += 1
            continue
        else:
            target_list.append(0)
            i += 1
            continue  

In [ ]:
# double check
target_list[:10]

In [ ]:
df.head(10)

In [ ]:
# Create target feature
data['target'] = target_list
display(data.info())
display(data.target.value_counts())

### 4.2.7 Upsample positive class

The data is very imbalanced, so I will upsample the positive class to help balance that out.

In [ ]:
from sklearn.utils import resample

# separate classes
data_minority = data[data['target']==1]
data_majority = data[data['target']==0]

# upsample minority
data_minority_upsampled = resample(data_minority, 
                                 replace=True,      # sample with replacement
                                 n_samples=1773,    # to match majority class
                                 random_state=2)    # reproducible results)

# combine upsampled results with majority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])

# check
data_upsampled.target.value_counts()


In [ ]:
# Save to file
data_upsampled.to_csv('data/processed/data_upsampled.csv')
data.to_csv('data/processed/data_final_w_target.csv')

## 4.3 Fix Column Data Types

In [ ]:
# For reloading data if I screw up the df below
#data = pd.read_csv('data/processed/data_upsampled.csv', 
#                   index_col=['customer','invoice_date'])

data = data_upsampled

In [ ]:
# In all that engineering we lost our data types
data['industry'] = data['industry'].astype('category')
data['contact_owner'] = data['contact_owner'].astype('category')
data['county'] = data['county'].astype('category')

data['first_sale'] = pd.to_datetime(data['first_sale'])

# Check
data.info()

### 4.3.1 One hot encode Categorical features

In [ ]:
data = pd.get_dummies(data.drop(['item','description'], axis=1))
data.head()

## 4.4 Preprocessing - Scaling, Test/Train split

In [ ]:
from sklearn.preprocessing import StandardScaler

# Define X,y
X = data.drop(['target','first_sale'], axis=1)
y = data.target

# Preprocessing
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# Split into training/testing sets
from sklearn.model_selection import train_test_split

y = y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state = 2)

In [ ]:
# Check shapes
print(X.shape)
print(X_train.shape)
print(X_test.shape)

# Step 5 - Modelling

Now to build out 3 different models.

## 5.1 - Model 1: Random Forest

Random forest is generally a good base model. It also has the added advantage of giving us feature importance, which can be translated to business impacts

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
model_1 = rfc.fit(X_train, y_train)
y_pred1 = model_1.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred1, digits=2))

### 5.1.1 - Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

# Random forest param tuning
param_grid = {
    'n_estimators':np.arange(100,500,100),
    'max_depth':[None, 3, 7, 11, 15],
    'min_samples_leaf':np.arange(3,7,2)
}

rfc = RandomForestClassifier()
gs1 = GridSearchCV(rfc, param_grid, verbose=5, cv=5, n_jobs=-1)

gs1.results = gs1.fit(X_train, y_train)

In [ ]:
print('Best parameters: {}'.format(gs1.results.best_params_))

In [ ]:
# Fit best model
rfc = RandomForestClassifier(max_depth=None,
                            min_samples_leaf=3,
                            n_estimators=400)
model1 = rfc.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred1, digits=2))

### 5.1.2 - RFC Feature Importances

For this model, want to see the top features of importance.

In [ ]:
# Print feature importances
for name, importance in zip(data.drop(['target','first_sale'], axis=1), 
                            model_1.feature_importances_):
    print(name, ': ', importance)

In [ ]:
feat_importances = pd.Series(model_1.feature_importances_, 
                             index=data.drop(['target','first_sale'], axis=1).columns)
feat_importances.sort_values(inplace=True, ascending=False)
feat_importances[:10].plot(kind='barh')

plt.title("Top 10 Feature Importances")
plt.savefig("figures/feature_importances.png")

## 5.2 - Model 2: Support Vector Machine (SVM)

SVM models are great in high dimensional space. Our data is relatively sparse due to the number of categorical features, so will be interesting to see how well this algorithm fits a model to our data.

In [ ]:
from sklearn.svm import SVC
svc = SVC()
model_2 = svc.fit(X_train, y_train)
y_pred2 = model_2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred2, digits=2))

### 5.2.1 - Hyperparameter Tuning

In [ ]:
# SVC parameter tuning
param_grid = {
    'kernel':['linear','rbf','poly'],
    'gamma':[0.1,1,10,100],
    'C':[0.1,1.0,10,100,1000]
}

gs2 = GridSearchCV(svc, param_grid, verbose=5, cv=5, n_jobs=-1)

gs2.results = gs2.fit(X_train, y_train)

## 5.3 - Model 3: Logistic Regression

Logistic regression is a regression algorithm used for classification.

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
model_3 = lr.fit(X_train, y_train)
y_pred3 = model_3.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred3, digits=2))

### 5.3.1 - Hyperparameter Tuning

In [ ]:
# Log Reg parameter tuning
param_grid = {
    'penalty':['l1','l2'],
    'C':[0.1,1.0,10,100,1000]
}

gs3 = GridSearchCV(lr, param_grid, verbose=5, cv=5, n_jobs=-1)

gs3.results = gs3.fit(X_train, y_train)